In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'P100CLIE'
file_name = 'P100CLIE'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/single-gpu/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/single-gpu/")

## Dense Layer

In [4]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfDense

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,32.784080,2037.468080,2042.244520,0.0,32.0,0.732888,0.701663,0.679047,0.019520,1.365308e+08,...,0.083280,0.083080,0.084800,0.493120,0.085440,0.085040,0.251960,0.249560,0.251560,0.246920
std,18.460976,1187.081709,1183.780913,0.0,0.0,0.395866,0.391203,0.392509,0.009891,1.597323e+08,...,0.276311,0.276009,0.278589,0.499963,0.279541,0.278947,0.434147,0.432767,0.433918,0.431228
min,1.000000,1.000000,1.000000,0.0,32.0,0.346479,0.326743,0.274577,0.000976,7.600000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,996.000000,1011.000000,0.0,32.0,0.456280,0.428758,0.402660,0.012755,2.204244e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,2043.000000,2035.000000,0.0,32.0,0.592036,0.558889,0.538108,0.017729,7.419462e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,3065.250000,3077.250000,0.0,32.0,0.845661,0.812736,0.792259,0.024069,1.949631e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,64.000000,4095.000000,4095.000000,0.0,32.0,3.094482,3.024440,2.880960,0.117104,1.031366e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [4]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,activation_fct,use_bias,optimizer,gpu,timeUsed_median,timeUsed_min,timeUsed_max,timeUsed_std
0,48,342,1,17,8,1,1,32,3,True,0,0,9.735322,9.656954,9.856997,0.066676
1,44,224,2,25,27,2,0,32,2,False,6,0,17.275147,17.233572,17.291608,0.020443
2,31,102,3,81,66,2,1,32,1,False,6,0,15.299087,15.251312,15.323405,0.026123
3,19,98,7,15,101,3,1,32,3,True,2,0,3.848872,3.819356,3.884783,0.022510
4,20,387,2,8,6,4,1,32,3,False,0,0,1.227965,1.189265,1.272049,0.033456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,35,45,6,209,184,1,1,32,0,True,0,1,7.942239,7.834379,7.964333,0.056774
1496,33,171,7,43,46,4,0,32,2,True,0,0,3.973818,3.850722,3.976393,0.058644
1497,32,191,2,26,36,3,1,32,3,True,0,1,1.572355,1.525044,1.572378,0.022308
1498,43,50,6,187,199,1,1,32,3,False,6,0,37.659001,37.622110,37.730122,0.044828


In [5]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [6]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [10]:
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
0,48,342,1,17,8,1,1,32,1,0,...,0,0,0,1,0,0,0,0,1,0
1,44,224,2,25,27,2,0,32,0,0,...,0,0,0,0,1,0,0,0,0,1
2,31,102,3,81,66,2,1,32,0,0,...,0,0,0,0,1,0,0,1,0,0
3,19,98,7,15,101,3,1,32,1,0,...,0,0,0,0,0,0,0,0,1,0
4,20,387,2,8,6,4,1,32,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,35,45,6,209,184,1,1,32,1,1,...,0,0,0,1,0,0,1,0,0,0
1496,33,171,7,43,46,4,0,32,1,0,...,0,0,0,1,0,0,0,0,0,1
1497,32,191,2,26,36,3,1,32,1,1,...,0,0,0,1,0,0,0,0,1,0
1498,43,50,6,187,199,1,1,32,0,0,...,0,0,0,0,1,0,0,0,1,0


In [9]:
dfConv.describe()

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.00000,5000.000000,...,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,32.485600,253.015200,3.949600,72.786800,72.054000,2.503200,0.502600,32.0,0.49940,0.250000,...,0.08680,0.072400,0.085800,0.508600,0.084000,0.081800,0.234000,0.238200,0.259400,0.268400
std,18.239713,147.295334,1.990943,351.119287,332.452391,1.114019,0.500043,0.0,0.50005,0.433056,...,0.28157,0.259175,0.280097,0.499976,0.277416,0.274087,0.423415,0.426025,0.438349,0.443171
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,32.0,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,124.750000,2.000000,10.000000,10.000000,2.000000,0.000000,32.0,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,32.000000,252.000000,4.000000,20.000000,19.000000,2.000000,1.000000,32.0,0.00000,0.000000,...,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,48.000000,378.000000,6.000000,40.000000,39.000000,3.000000,1.000000,32.0,1.00000,0.250000,...,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,64.000000,512.000000,7.000000,9895.000000,8208.000000,4.000000,1.000000,32.0,1.00000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
dfConv.columns

Index(['batchsize', 'matsize', 'kernelsize', 'channels_in', 'channels_out',
       'strides', 'padding', 'precision', 'use_bias', 'gpu', 'timeUsed_median',
       'timeUsed_min', 'timeUsed_max', 'timeUsed_std', 'elements_matrix',
       'elements_kernel', 'ops', 'memory_weights', 'memory_in', 'memory_out',
       'opt_Adadelta', 'opt_Adagrad', 'opt_Adam', 'opt_Momentum', 'opt_None',
       'opt_RMSProp', 'opt_SGD', 'act_None', 'act_relu', 'act_sigmoid',
       'act_tanh'],
      dtype='object')

In [24]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))